# Facial Keypoint Detection

**Explorations on facial image keypoint detection.**  

This project contains the explorations on face feature detection (for the [Kaggle competition](https://www.kaggle.com/c/facial-keypoints-detection)), as part of the final project for the 207-Machine Learning course for [Berkeley's Master in Information and Data Science](https://datascience.berkeley.edu/).  

The project was developed by: [Alex](https://github.com/keivahn), [Ankit](https://github.com/ankittharwani), [Nina](https://github.com/kuknina) and [Will](https://github.com/willahscott).  


## Presentation
We created a [Prezi adventure](https://prezi.com/an_apyqfaeaj/recognizing-key-facial-points/#) to showcase our ideas and explorations.

## Contents of this notebook

* [GitHub Repo](#GitHub-Repo)  
* [Preprocessing](#Preprocessing)  
    * [Face Detection](#Face-Detection)  
    * [Dimensionality Reduction](#Dimensionality-Reduction)  
        * [PCA](#PCA)  
        * [GMM](#GMM)  
    * [Image Processing](#Image-Processing)  
        * [Edge Detection](#Edge-Detection)  
        * [Noise Reduction](#Noise-Reduction)  
        * [Finding Contours](#Finding-Contours)  
* [Models Explored](#Models-Explored)  
    * [Baseline - KNearest Neighbors](#Baseline---KNearest-Neighbors)  
    * [Neural Nets](#Neural-Nets)  
        * [Convolutional Neural Network](#Convolutional-Neural-Network)
    * [Bayes Classifiers](#Bayes-Classifiers)  
    * [Logistic Regression](#Logistic-Regression)  
    * [Trees](#Trees)  
    * [Bayes Classifiers](#Bayes Classifiers)  
    * [Bayes Classifiers](#Bayes Classifiers)  
    * [Bayes Classifiers](#Bayes Classifiers)  
* [Final Model](#Final-Model)  
* [Conclusions and Remarks](#Conclusions-and-Remarks)  
* [Appendix](#Appendix)  


## GitHub Repo  

Given that we were 4 team members and we were planning on doing a more breadth analysis, we created a [GitHub repository](https://github.com/WillahScott/facial-keypoint-detection) with a structure that would make our work **independent** but **compatible** as well a set of common `tools` that would create our explorations cleaner and more efficient.

The structure we came up is as follows:  

* **`scripts/`**  

	* **`tools/`** contains a set of additional tools that make the exploration and modeling cleaner.
        * `submit.py` - functions for generating the Kaggle submission files  
        * `getdata.py` - loading and preprocessing the datasets
        * `save4later.py` - basic pickler for storing and retrieving (and listing the available) models, as well as preprocessed dataset
    
	* **`explorations/`** contains dataset explorations, plots and initial modeling attempts  

    * **`preprocessors/`** contains the preprocessing scripts

	* **`modelers/`** contains all serious modeling attempts, including the generation of their submission files
    

* **`data/`**  

	* **`datasets/`** contains the original kaggle data  

	* **`preprocessed/`** contains preprocessed datasets. For *temporal* time-consuming preprocessings   

	* **`models/`** contains the persistent storages of the models created. Each pickled model contains: name, alias, description, model-object, prediction-df, [training-time], [predicting-time]  

    * **`submissions/`** contains the csv files submitted to the Kaggle competition  



## Goal

Bla bla


## Preprocessing

### Face Detection - Background Filtering  

#### OpenCV

In [ ]:
import cv2

#### Haar Classifier

In [1]:
# Haar classifier code

### Dimensionality Reduction  

In [ ]:
from sklearn.decomposition import PCA
from sklearn.mixture import GMM

#### PCA

#### GMM

### Image Processing   

#### Edge Detection

#### Noise Reduction

#### Finding Contours

## Models Explored

### Baseline - KNearest Neighbors

### Neural Nets

#### Convolutional Neural Network  

Given that a great number of the latest (and best) image detection algorithms make use of Convolutional Neural Networks, we found it necessary to explore this methodology.  

Following [Daniel Nourian's tutorial](http://danielnouri.org/notes/2014/12/17/using-convolutional-neural-nets-to-detect-facial-keypoints-tutorial/) we ...

For this deep learning algorithm we did not use any preprocessings, since the "deep" nature already acts as a features engineer, and also for comparison purposes (would the CNN find better features than if we used usual computer vision tools for feature engineering?).  


### Bayes Classifiers

### Logistic Regression

### Trees 

#### Random Forests

### AdaBoost ?

## Final Model

## Conclusions and Remarks

Bla bla


## Appendix